In [1]:
import os
import re
from glob import glob
from functions import *
from Levenshtein import distance

In [2]:
doc = 'Scrape_and_some_views/raw_data_old/Serie A/2013/Game 230.csv'
extract_participations_1(doc)

[['In', 'Out', 'ID', 'Club'],
 ['00:00 - 1T', '45:00 - 2T', '142720', 'Santos / SP'],
 ['00:00 - 1T', '45:00 - 2T', '303187', 'Criciúma / SC'],
 ['00:00 - 1T', '45:00 - 2T', '136752', 'Santos / SP'],
 ['00:00 - 1T', '45:00 - 2T', '295880', 'Criciúma / SC'],
 ['00:00 - 1T', '45:00 - INT', '445840', 'Santos / SP'],
 ['00:00 - 1T', '45:00 - 2T', '175475', 'Criciúma / SC'],
 ['00:00 - 1T', '45:00 - 2T', '185679', 'Santos / SP'],
 ['00:00 - 1T', '45:00 - 2T', '165264', 'Criciúma / SC'],
 ['00:00 - 1T', '45:00 - 2T', '161941', 'Santos / SP'],
 ['00:00 - 1T', '45:00 - 2T', '163885', 'Criciúma / SC'],
 ['00:00 - 1T', '45:00 - 2T', '186450', 'Santos / SP'],
 ['00:00 - 1T', '45:00 - 2T', '330478', 'Criciúma / SC'],
 ['00:00 - 1T', '45:00 - 2T', '185339', 'Santos / SP'],
 ['00:00 - 1T', '45:00 - 2T', '176375', 'Criciúma / SC'],
 ['00:00 - 1T', '29:00 - 2T', '322398', 'Santos / SP'],
 ['00:00 - 1T', '38:00 - 2T', '181803', 'Criciúma / SC'],
 ['00:00 - 1T', '45:00 - 2T', '164974', 'Santos / SP'],
 

In [3]:
games1 = sorted(glob('Scrape_and_some_views/raw_data_old/*/*/Game *'))
games2 = sorted(glob('Scrape_and_some_views/raw_data/*/*/Game *'))

games = []
for game in games1:
    aux = game.replace('raw_data_old', 'raw_data')
    if aux in games2:
        games.append([game, aux])
    else:
        games.append([game])
        
for game in games2:
    aux = game.replace('raw_data', 'raw_data_old')
    if aux not in games1:
        games.append([game])

In [4]:
players = []
c1 = []
c2 = []
for game in games:
    for doc in game:
        game_players = extract_players(doc)
        if len(game_players) == 0:
            c1.append(doc)
            
        if len(game_players) < 22:
            c2.append(doc)

        players += game_players
            
len(c1), len(c2)

(13133, 13139)

In [5]:
all_players = {}
for player in players:
    if player[0] in all_players:
        if player[-1] not in all_players[player[0]]['Clubs']:
            all_players[player[0]]['Clubs'].append(player[-1])
    else:
        all_players[player[0]] = {'Name' : player[2], 'Type' : player[3], 'Clubs' : [player[-1]]}

In [6]:
c0 = 0
c1 = []
c2 = []
for game in games:
    for doc in game:
        c0 += 1
        try:
            extract_participations_1(doc)
        except ValueError:
            try:
                extract_participations_2(doc)
            except ValueError:    
                c1 += [doc]
                
        except IndexError:
            c2 += [doc]

c0, len(c1), len(c2)

(23449, 21, 97)

In [7]:
for game in c2:
    if 'Serie D' not in game:
        print(game)

Scrape_and_some_views/raw_data_old/Serie B/2013/Game 037.csv


In [8]:
# buscar gols e placares

In [9]:
def extract_goal_info(line):
    print(line)
    if ',' not in line:
        time = re.findall('\dT', line)
        sep = line.find(time[0]) + 2
        time = line[:sep]
        line = line[sep:]
        k = 1
        while line[:k].isdigit():
            number = line[:k]
            k += 1

        line = line[k - 1:]
        tp = line[:2]
        line = re.sub(' ', '', line[2:])

        return time, number, tp, line
    
    time = re.findall('\dT', line)
    sep = line.find(time[0]) + 2
    time = re.sub(',', '', line[:sep])
    line = line[sep:]
    line = line[1:].split(',')
    number = line[0]
    tp = line[1]
    club = line[-1]
    
    return time, number, tp, club

In [10]:
def find_player(players, number, club):
    for player in players:
        if player[1] == number and club == re.sub(' / ', '/', player[-1]):
            return player[0]

In [11]:
def extract_score(game):
    file = open(game, encoding = 'ISO-8859-1')
    data = file.readlines()
    file.close()
    if data == []:
        return ''
    
    goals = []
    for line in data:
        score = re.findall('\dx\d', re.sub(' ', '', line.lower()))
        if score != []:
            for i in range(len(score)):
                score[i] = re.sub('x', ' x ', score[i])

            score = score[-1]
            break
            
    return score

    '''
    players = extract_players(game)
    club1, club2 = '', ''
    for player in players:
        if club1 == '':
            club1 = re.sub(' / ', '/', player[-1])
        elif club2 == '' and player[-1] != club1:
            club2 = re.sub(' / ', '/', player[-1])
            break

    flag1 = False
    flag2 = False
    for line in data:
        if flag2 and not line[0].isdigit():
            break
        if flag2:
            time, number, tp, club = extract_goal_info(line.strip())
            print(club)
            if re.sub(' ', '', club1) in club:
                club = club1
            else:
                club = club2
            
            print(club)
            print()
            player = find_player(players, number, club)
            goals.append([time, player, tp, club])
        elif flag1:
            flag2 = True
        elif 'Gols' in line:
            flag1 = True

    return score, goals
    '''

game = 'Scrape_and_some_views/raw_data/Copa do Brasil/2017/Game 019.csv'
extract_score(game)

'0 x 2'

In [12]:
# ao extrair os gols de cada partida será preciso ver
# o problema do guarani de juazeiro/ce -> guarani/ce
# entre outros

In [13]:
def extract_info(game):
    try:
        players = extract_participations_1(game)
    except:
        try:
            players = extract_participations_2(game)
        except:
            return None
        
    if players == [['In', 'Out', 'ID', 'Club']]:
        return None
    else:
        score = extract_score(game)
        
    return players, score

c1, c2 = 0, 0
for game in games:
    for doc in game:
        info = extract_info(doc)
        if info == None:
            c1 += 1
        else:
            c2 += 1
            
c1, c2

(12639, 10810)

In [14]:
extract_info('Scrape_and_some_views/raw_data_old/Serie D/2020/Game 517.csv')

([['In', 'Out', 'ID', 'Club'],
  ['00:00 - 1T', '45:00 - 2T', '177953', 'Floresta / CE'],
  ['00:00 - 1T', '45:00 - 2T', '352377', 'Mirassol / SP'],
  ['00:00 - 1T', '36:00 - 2T', '298633', 'Floresta / CE'],
  ['00:00 - 1T', '45:00 - 2T', '440110', 'Mirassol / SP'],
  ['00:00 - 1T', '45:00 - 2T', '298916', 'Floresta / CE'],
  ['00:00 - 1T', '45:00 - 2T', '456006', 'Mirassol / SP'],
  ['00:00 - 1T', '45:00 - 2T', '385488', 'Floresta / CE'],
  ['00:00 - 1T', '45:00 - 2T', '184109', 'Mirassol / SP'],
  ['00:00 - 1T', '45:00 - 2T', '177085', 'Floresta / CE'],
  ['00:00 - 1T', '45:00 - 2T', '338246', 'Mirassol / SP'],
  ['00:00 - 1T', '45:00 - 2T', '187325', 'Floresta / CE'],
  ['00:00 - 1T', '29:00 - 2T', '313582', 'Mirassol / SP'],
  ['00:00 - 1T', '22:00 - 2T', '532888', 'Floresta / CE'],
  ['00:00 - 1T', '45:00 - 2T', '403332', 'Mirassol / SP'],
  ['00:00 - 1T', '45:00 - 2T', '172261', 'Floresta / CE'],
  ['00:00 - 1T', '29:00 - 2T', '177093', 'Mirassol / SP'],
  ['00:00 - 1T', '45:00 -